In [0]:
!pip install tensorflow==2.0




In [0]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [0]:
from google.colab import files
import os
import random

# TensorFlow and tf.keras
#import tensorflow as tf
#from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd

# Sklearn
from sklearn.model_selection import train_test_split # Helps with organizing data for training
from sklearn.metrics import confusion_matrix # Helps present results as a confusion-matri

In [0]:
!unzip /content/drive/My\ Drive/data/leapGestRecog.zip


In [0]:
# Unzip your data set here 
!rm -R data/
!unzip data-4.zip

In [0]:

imagepaths = []

# Go through all the files and subdirectories inside a folder and save path to images inside list
for root, dirs, files in os.walk("./data", topdown=False): 
  for name in files:
    path = os.path.join(root, name)
    if path.endswith("png"): # We want only the images
      imagepaths.append(path)

print(len(imagepaths)) # If > 0, then a PNG image was loaded
random.shuffle(imagepaths)

7000


We first extract all the paths of the .png files and then shuffle it

In [0]:
# This function is used more for debugging and showing results later. It plots the image into the notebook

def plot_image(path):
  img = cv2.imread(path) # Reads the image into a numpy.array
  img_cvt = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Converts into the corret colorspace (RGB)
  img_cvt = cv2.resize(img_cvt , (300,300))
  print(img_cvt.shape) # Prints the shape of the image just to check
  plt.grid(False) # Without grid so we can see better
  plt.imshow(img_cvt) # Shows the image
  plt.xlabel("Width")
  plt.ylabel("Height")
  plt.title("Image " + path)


This function is for debuging and checking how the image looks before training

In [0]:
X = [] # Image data
y = [] # Labels

# Loops through imagepaths to load images and labels into arrays
for path in imagepaths:
  img = cv2.imread(path) # Reads image and returns np.array
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Converts into the corret colorspace (GRAY)
  img = cv2.resize(img, (300, 300)) # Reduce image size so training can be faster
  X.append(img)

  # Processing label in image path
  category = path.split("/")[3]
  label = int(category.split("_")[1]) # We need to convert 10_down to 00_down, or else it crashes
  y.append(label)

# Turn X and y into np.array to speed up train_test_split
X = np.array(X, dtype="float16")
print(X.shape)
X = X.reshape(len(imagepaths), 300, 300, 1) # Needed to reshape so CNN knows it's different images
y = np.array(y)

print("Images loaded: ", len(X))
print("Labels loaded: ", len(y))



(7000, 300, 300)
Images loaded:  7000
Labels loaded:  7000


We preprocess the image and then create two arrays X and Y with the image and labels respectively 

In [0]:
ts = 0.3 # Percentage of images that we want to use for testing. The rest is used for training.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ts, random_state=42)

We then split the dataSet intpo Training and Testing sets 

In [0]:
# Construction of model

layers =[
         tf.keras.layers.Conv2D(32, (5, 5), activation='relu', input_shape=(300, 300, 1)),
         tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(2,2)),
         tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
         tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(2,2)),
         tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
         tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(2,2)),
         tf.keras.layers.Flatten(),
         tf.keras.layers.Dense(128, activation='relu'),
         tf.keras.layers.Dense(7, activation='softmax')
]

model = tf.keras.Sequential(layers)
model.compile(optimizer=tf.optimizers.Adam(),
              loss = tf.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.metrics.SparseCategoricalAccuracy()])
# Trains the model for a given number of epochs (iterations on a dataset) and validates it.
model.fit(X_train, y_train, epochs=5, batch_size=64, verbose=2, validation_data=(X_test, y_test))
model.save('custModelOrgRes-9.h5')


Train on 4900 samples, validate on 2100 samples
Epoch 1/5
4900/4900 - 897s - loss: 10.1494 - sparse_categorical_accuracy: 0.8749 - val_loss: 0.0531 - val_sparse_categorical_accuracy: 0.9862
Epoch 2/5
4900/4900 - 905s - loss: 0.0149 - sparse_categorical_accuracy: 0.9980 - val_loss: 0.0411 - val_sparse_categorical_accuracy: 0.9890
Epoch 3/5
4900/4900 - 899s - loss: 0.0112 - sparse_categorical_accuracy: 0.9990 - val_loss: 0.0303 - val_sparse_categorical_accuracy: 0.9943
Epoch 4/5
4900/4900 - 909s - loss: 0.0092 - sparse_categorical_accuracy: 0.9994 - val_loss: 0.0281 - val_sparse_categorical_accuracy: 0.9943
Epoch 5/5
4900/4900 - 907s - loss: 0.0094 - sparse_categorical_accuracy: 0.9994 - val_loss: 0.0278 - val_sparse_categorical_accuracy: 0.9948


The ML model which currently takes the images with a res of 300 x 300.

On training for 5 epochs and adam optimizer we get an accuracy of 99.48% 

In [0]:
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test accuracy: {:2.2f}%'.format(test_acc*100))